In [1]:
import requests as req
from bs4 import BeautifulSoup
import csv 
import matplotlib
import pandas as pd
import numpy as np
import json
from io import StringIO
import csv
import os
import time



In [6]:
def getStatesList(typeList):
  #the url of the site I'm scraping    
  stateListUrl = "https://inclinedesigngroup.com/resource-lists-of-states-for-easy-form-building/"
  # use requests to get the page and then pass in the page content. 
  page = req.get(stateListUrl, timeout = 1)
  #create the soup objec
  soup = BeautifulSoup(page.content, 'html.parser')
  #find all the <p> tags in the webpage
  pTags = soup.find_all('p')
  for tag in pTags :
    #print(tag.text)

    if(len(tag.text) >=3 ):
      #print(tag.text[0])
      #print(tag.text[3])
      if(typeList == "fullName"):
        if((tag.text[0]=="“") & (tag.text[2] == "l")):
          #replacing the weird utf " with nothing and splitting them into their states
          return tag.text.replace("“", "").replace("”", "").replace(", ",",").replace(" ","-").split(",")
states = getStatesList("fullName")
#for state in states:
  #print(state)

In [56]:
#<p class="main" data-bind="tabValue: [value, filter, old]" style="display: block;">$10.34</p>
#mydivs = soup.find_all("div", {"class": "stylelistrow"})
# <div data-bind="itemCSS: slug, callout: slug, click: $root.tabClick" class="item-min-wage bm-callout active">
def getStateMinimumWage(states):
        stateLits = []
        url = 'https://www.epi.org/minimum-wage-tracker/#/min_wage/' + state
        print(url)
        page = req.get(url, timeout = 1)
        soup = BeautifulSoup(page.content, 'html.parser')
        #print(soup.prettify)
        tags = soup.find_all("tr")
        for tag in tags:
            print(tag.text)
            stateLits.append(tag.text)
        return stateLits
stateList = getStateMinimumWage(states)

https://www.epi.org/minimum-wage-tracker/#/min_wage/Wyoming

[state] State
[locality] Locality
[min_wage] Minimum wage*
[min_wage last_inc] Most recent increase
[min_wage up_inc] Upcoming increases
[min_wage indexing] Indexing
[min_wage last_change] Most recent major change to minimum wage law
[min_wage notes] Notes
[tip_wage] Subminimum wage for tipped workers / Tipped wage*
[tip_wage last_inc] Most recent increase
[tip_wage up_inc] Upcoming increases
[tip_wage last_change] Most recent major change to tipped wage law
[tip_wage notes] Notes


Alabama
 
 
 
 
 
 
 
 
 
 
 
 


Alabama
Birmingham
$7.25
 
$8.50, effective 7-1-2016* / $10.10, effective 7-1-2017 / Annual indexing beginning 7-1-2018
Annual increases based on “the increase, if any, in the cost of living.”
Birmingham enacted its own minimum wage in 2015 by city council ordinance.
* The Alabama state legislature passed a law in March 2016, shortly before the ordinance was to go into effect, preempting local governments from ado

In [60]:
stateList.pop(0)
stateList.pop(0)



Alabama
Birmingham
$7.25
 
$8.50, effective 7-1-2016* / $10.10, effective 7-1-2017 / Annual indexing beginning 7-1-2018
Annual increases based on “the increase, if any, in the cost of living.”
Birmingham enacted its own minimum wage in 2015 by city council ordinance.
* The Alabama state legislature passed a law in March 2016, shortly before the ordinance was to go into effect, preempting local governments from adopting minimum wages above the state’s minimum wage. 
 
 
$4.25, effective 7-1-2016* / $5.05, effective 7-1-2017 / Annual indexing beginning 7-1-2018
Birmingham enacted its own tipped wage in 2015 by city council ordinance.
* The Alabama state legislature passed a law in March 2016, shortly before the ordinance was to go into effect, preempting local governments from adopting minimum wages above the state’s minimum wage. The city ordinance sets a tipped minimum wage at 50% of the city’s regular minimum wage. 


Alaska
 
$10.34
$10.19 to $10.34, effective 1-1-2021
Annual indexi

In [81]:
f = open('statesMinimumWages.csv', 'w')
writer = csv.writer(f)
header = ['state', 'wage']
writer.writerow(header)
for state in stateList:
    #print(state)
    lines = state.split("\n")
    #for line in lines:
     #   print("l:" + line)
    if( len(lines) >=3):
        state = lines[1]
        pay = lines[3]
        print(lines[1] +", " +  lines[3])
        row = [state, pay]
        writer.writerow(row)
f.close()

Alabama, $7.25
Alaska, $10.34
Arizona, $12.80
Arizona, $15.50
Arkansas, $11.00
California, $15.00
California, $15.00
California, $16.20
California, $16.32
California, $16.40
California, $15.60
California, $16.37
California, $17.13
California, $15.25
California, $15.56
California, $16.40
California, $15.00
California, $15.00
California, $15.00
California, $15.65
California, $17.10
California, $15.77
California, $15.06
California, $16.45
California, $15.00
California, $15.85
California, $16.20
California, $15.54
California, $15.00
California, $16.32
California, $16.20
California, $15.00
California, $16.20
California, $16.40
California, $15.00
California, $15.85
California, $16.00
California, $17.10
Colorado, $12.56
Colorado, $15.87
Connecticut, $13.00
Delaware, $10.50
Washington D.C., $15.20
Florida, $10.00
Georgia, $5.15
Hawaii, $10.10
Idaho, $7.25
Illinois, $12.00
Illinois, $15.00
Illinois, $13.00
Indiana, $7.25
Iowa, $7.25
Iowa, $7.25
Iowa, $7.25
Iowa, $7.25
Iowa, $7.25
Kansas, $7.25


In [103]:
df = pd.read_csv('statesMinimumWages.csv')
df = df.drop_duplicates(keep = "first", subset = 'state')
def removeDollar(string):
    string = string.replace("$", "")
    if(len(string) == 1):
        string = 7.25
    return string
df['wage'] = df['wage'].apply(removeDollar)
df.fillna(7.25, inplace=True)
df.head(100)


,state,wage
0,Alabama,7.25
1,Alaska,10.34
2,Arizona,12.80
4,Arkansas,11.00
5,California,15.00
38,Colorado,12.56
40,Connecticut,13.00
41,Delaware,10.50
42,Washington D.C.,15.20
43,Florida,10.00


In [106]:
df.to_csv("statesMinimumWages.csv", index = False)